## 1. Basic Exception Logging

In [1]:
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

# Method 1: exc_info=True (includes full traceback)
try:
    result = 10 / 0
except ZeroDivisionError as e:
    logger.error("Division by zero error", exc_info=True)

# Method 2: exception() shorthand
try:
    value = int("not_a_number")
except ValueError:
    logger.exception("Failed to convert string to integer")

2025-12-24 06:57:40,315 - __main__ - ERROR - Division by zero error
Traceback (most recent call last):
  File "C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_31800\442219821.py", line 12, in <module>
    result = 10 / 0
             ~~~^~~
ZeroDivisionError: division by zero
2025-12-24 06:57:40,316 - __main__ - ERROR - Failed to convert string to integer
Traceback (most recent call last):
  File "C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_31800\442219821.py", line 18, in <module>
    value = int("not_a_number")
ValueError: invalid literal for int() with base 10: 'not_a_number'


## 2. Custom Exception Classes

In [2]:
import logging
from typing import Optional

logger = logging.getLogger(__name__)

class ApplicationError(Exception):
    """Base application error with logging"""
    
    def __init__(self, message: str, error_code: Optional[str] = None, **context):
        self.message = message
        self.error_code = error_code or "UNKNOWN"
        self.context = context
        super().__init__(self.message)
    
    def log_error(self, logger, level=logging.ERROR):
        """Log the error with context"""
        log_message = f"[{self.error_code}] {self.message}"
        if self.context:
            log_message += f" | Context: {self.context}"
        logger.log(level, log_message, exc_info=True)

class DatabaseError(ApplicationError):
    """Database-specific error"""
    pass

class ValidationError(ApplicationError):
    """Validation error"""
    pass

class APIError(ApplicationError):
    """API error with status code"""
    
    def __init__(self, message: str, status_code: int, **context):
        self.status_code = status_code
        super().__init__(message, error_code=f"API_{status_code}", **context)

# Usage
try:
    raise ValidationError(
        "Invalid email format",
        error_code="INVALID_EMAIL",
        email="user@example",
        field="email"
    )
except ValidationError as e:
    e.log_error(logger)

2025-12-24 06:57:40,395 - __main__ - ERROR - [INVALID_EMAIL] Invalid email format | Context: {'email': 'user@example', 'field': 'email'}
Traceback (most recent call last):
  File "C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_31800\77244645.py", line 39, in <module>
    raise ValidationError(
    ...<4 lines>...
    )
ValidationError: Invalid email format


## 3. Error Context Management

In [3]:
import logging
import traceback
from typing import Dict, Any

logger = logging.getLogger(__name__)

class ErrorContext:
    """Manage error context and details"""
    
    def __init__(self, operation: str, **context):
        self.operation = operation
        self.context = context
        self.error_details = {}
    
    def log_error(self, exception: Exception, logger):
        """Log error with full context"""
        self.error_details = {
            'operation': self.operation,
            'exception_type': type(exception).__name__,
            'exception_message': str(exception),
            'traceback': traceback.format_exc(),
            'context': self.context
        }
        
        logger.error(
            f"Error in {self.operation}: {str(exception)}",
            extra=self.error_details,
            exc_info=True
        )
    
    def get_error_summary(self) -> Dict[str, Any]:
        """Get summary of error for API response"""
        return {
            'error': True,
            'operation': self.operation,
            'message': self.error_details.get('exception_message', 'Unknown error'),
            'type': self.error_details.get('exception_type', 'Unknown')
        }

# Usage
ctx = ErrorContext(
    'database_query',
    user_id=123,
    query='SELECT * FROM users'
)

try:
    raise ConnectionError("Database connection failed")
except ConnectionError as e:
    ctx.log_error(e, logger)
    error_response = ctx.get_error_summary()
    print("Error Response:", error_response)

2025-12-24 06:57:40,416 - __main__ - ERROR - Error in database_query: Database connection failed
Traceback (most recent call last):
  File "C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_31800\1537875061.py", line 48, in <module>
    raise ConnectionError("Database connection failed")
ConnectionError: Database connection failed


Error Response: {'error': True, 'operation': 'database_query', 'message': 'Database connection failed', 'type': 'ConnectionError'}


## 4. Try-Except-Finally Patterns

In [4]:
import logging
import tempfile
import os

logger = logging.getLogger(__name__)

# Pattern 1: Resource cleanup with logging
def read_file_with_logging(filename: str) -> str:
    """Read file with proper error handling and cleanup"""
    file_handle = None
    try:
        logger.debug(f"Opening file: {filename}")
        file_handle = open(filename, 'r')
        content = file_handle.read()
        logger.info(f"Successfully read {len(content)} bytes from {filename}")
        return content
        
    except FileNotFoundError:
        logger.error(f"File not found: {filename}")
        raise
        
    except IOError as e:
        logger.error(f"IO error reading {filename}: {e}", exc_info=True)
        raise
        
    finally:
        if file_handle:
            logger.debug(f"Closing file: {filename}")
            file_handle.close()

# Pattern 2: Using context manager
import contextlib

@contextlib.contextmanager
def logged_operation(operation_name: str):
    """Context manager for logging operation lifecycle"""
    try:
        logger.info(f"Starting operation: {operation_name}")
        yield
        logger.info(f"Completed operation: {operation_name}")
    except Exception as e:
        logger.error(f"Failed operation: {operation_name}", exc_info=True)
        raise
    finally:
        logger.debug(f"Cleanup for operation: {operation_name}")

# Usage
with logged_operation("data_processing"):
    logger.debug("Processing data...")
    # Simulate work
    logger.info("Data processed successfully")

2025-12-24 06:57:40,457 - __main__ - INFO - Starting operation: data_processing
2025-12-24 06:57:40,458 - __main__ - DEBUG - Processing data...
2025-12-24 06:57:40,459 - __main__ - INFO - Data processed successfully
2025-12-24 06:57:40,459 - __main__ - INFO - Completed operation: data_processing
2025-12-24 06:57:40,460 - __main__ - DEBUG - Cleanup for operation: data_processing


## 5. Retry Logic with Logging

In [5]:
import logging
import time
from functools import wraps
from typing import Callable, Type

logger = logging.getLogger(__name__)

def retry_with_logging(
    max_attempts: int = 3,
    delay: int = 1,
    backoff: float = 2.0,
    exceptions: tuple = (Exception,)
):
    """Decorator for retry logic with logging"""
    
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            current_delay = delay
            
            for attempt in range(1, max_attempts + 1):
                try:
                    logger.debug(
                        f"Attempt {attempt}/{max_attempts} for {func.__name__}"
                    )
                    return func(*args, **kwargs)
                    
                except exceptions as e:
                    if attempt == max_attempts:
                        logger.error(
                            f"All {max_attempts} attempts failed for {func.__name__}: {e}",
                            exc_info=True
                        )
                        raise
                    
                    logger.warning(
                        f"Attempt {attempt} failed: {e}. "
                        f"Retrying in {current_delay}s..."
                    )
                    time.sleep(current_delay)
                    current_delay *= backoff
        
        return wrapper
    
    return decorator

# Usage
@retry_with_logging(max_attempts=3, delay=0.5, backoff=1.5)
def unstable_network_call():
    """Simulates an unstable network call"""
    import random
    if random.random() < 0.7:
        raise ConnectionError("Network timeout")
    return "Success"

try:
    result = unstable_network_call()
    logger.info(f"Operation successful: {result}")
except Exception as e:
    logger.error(f"Operation failed after retries: {e}")

2025-12-24 06:57:40,493 - __main__ - DEBUG - Attempt 1/3 for unstable_network_call
2025-12-24 06:57:40,494 - __main__ - WARNING - Attempt 1 failed: Network timeout. Retrying in 0.5s...
2025-12-24 06:57:40,996 - __main__ - DEBUG - Attempt 2/3 for unstable_network_call
2025-12-24 06:57:40,998 - __main__ - WARNING - Attempt 2 failed: Network timeout. Retrying in 0.75s...
2025-12-24 06:57:41,748 - __main__ - DEBUG - Attempt 3/3 for unstable_network_call
2025-12-24 06:57:41,750 - __main__ - ERROR - All 3 attempts failed for unstable_network_call: Network timeout
Traceback (most recent call last):
  File "C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_31800\3502328281.py", line 26, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_31800\3502328281.py", line 53, in unstable_network_call
    raise ConnectionError("Network timeout")
ConnectionError: Network timeout
2025-12-24 06:57:41,751 - __main__ - ERROR - Operation failed after retries: Net

## 6. Error Aggregation and Reporting

import logging
from typing import List, Dict
from dataclasses import dataclass
from datetime import datetime

logger = logging.getLogger(__name__)

@dataclass
class ErrorRecord:
    timestamp: datetime
    operation: str
    error_type: str
    message: str
    severity: str

class ErrorAggregator:
    """Aggregate and track errors for reporting"""
    
    def __init__(self):
        self.errors: List[ErrorRecord] = []
        self.error_counts: Dict[str, int] = {}
    
    def record_error(
        self,
        operation: str,
        exception: Exception,
        severity: str = "ERROR"
    ):
        """Record an error"""
        error_record = ErrorRecord(
            timestamp=datetime.now(),
            operation=operation,
            error_type=type(exception).__name__,
            message=str(exception),
            severity=severity
        )
        
        self.errors.append(error_record)
        
        # Update counts
        error_key = f"{operation}:{error_record.error_type}"
        self.error_counts[error_key] = self.error_counts.get(error_key, 0) + 1
        
        # Log the error
        logger.log(
            getattr(logging, severity),
            f"[{operation}] {error_record.error_type}: {error_record.message}"
        )
    
    def get_report(self) -> Dict:
        """Generate error report"""
        total_errors = len(self.errors)
        critical = len([e for e in self.errors if e.severity == "CRITICAL"])
        errors = len([e for e in self.errors if e.severity == "ERROR"])
        warnings = len([e for e in self.errors if e.severity == "WARNING"])
        
        return {
            'total_errors': total_errors,
            'critical': critical,
            'errors': errors,
            'warnings': warnings,
            'error_types': self.error_counts,
            'recent_errors': [
                {
                    'timestamp': str(e.timestamp),
                    'operation': e.operation,
                    'type': e.error_type,
                    'message': e.message
                }
                for e in self.errors[-5:]  # Last 5 errors
            ]
        }

# Usage
aggregator = ErrorAggregator()

try:
    raise ValueError("Invalid input")
except ValueError as e:
    aggregator.record_error("validation", e)

try:
    raise ConnectionError("Database unavailable")
except ConnectionError as e:
    aggregator.record_error("database", e, severity="CRITICAL")

report = aggregator.get_report()
import json
print(json.dumps(report, indent=2))

## 7. Structured Error Logging

In [6]:
import logging
import json
from typing import Any, Dict

logger = logging.getLogger(__name__)

class StructuredErrorFormatter(logging.Formatter):
    """Format errors as structured JSON for log aggregation tools"""
    
    def format(self, record: logging.LogRecord) -> str:
        log_data = {
            'timestamp': self.formatTime(record),
            'level': record.levelname,
            'logger': record.name,
            'message': record.getMessage(),
            'function': record.funcName,
            'line': record.lineno,
        }
        
        if record.exc_info:
            log_data['exception'] = {
                'type': record.exc_info[0].__name__,
                'message': str(record.exc_info[1]),
                'traceback': self.formatException(record.exc_info)
            }
        
        # Include extra fields
        for key, value in record.__dict__.items():
            if key not in ['name', 'msg', 'args', 'created', 'filename',
                          'funcName', 'levelname', 'levelno', 'lineno',
                          'module', 'msecs', 'message', 'pathname', 'process',
                          'processName', 'relativeCreated', 'thread', 'threadName',
                          'exc_info', 'exc_text', 'stack_info']:
                log_data[key] = str(value)
        
        return json.dumps(log_data)

# Setup structured logging
handler = logging.StreamHandler()
handler.setFormatter(StructuredErrorFormatter())

structured_logger = logging.getLogger('structured')
structured_logger.addHandler(handler)
structured_logger.setLevel(logging.DEBUG)

# Usage
try:
    raise ValueError("Invalid configuration")
except ValueError as e:
    structured_logger.error(
        "Configuration validation failed",
        exc_info=True,
        extra={'config_file': '/etc/app.conf', 'user_id': 123}
    )

{"timestamp": "2025-12-24 06:57:41,764", "level": "ERROR", "logger": "structured", "message": "Configuration validation failed", "function": "<module>", "line": 50, "exception": {"type": "ValueError", "message": "Invalid configuration", "traceback": "Traceback (most recent call last):\n  File \"C:\\Users\\jdamodhar\\AppData\\Local\\Temp\\ipykernel_31800\\2151149838.py\", line 48, in <module>\n    raise ValueError(\"Invalid configuration\")\nValueError: Invalid configuration"}, "taskName": "Task-2", "config_file": "/etc/app.conf", "user_id": "123"}
2025-12-24 06:57:41,764 - structured - ERROR - Configuration validation failed
Traceback (most recent call last):
  File "C:\Users\jdamodhar\AppData\Local\Temp\ipykernel_31800\2151149838.py", line 48, in <module>
    raise ValueError("Invalid configuration")
ValueError: Invalid configuration


## 8. Error Recovery Patterns

import logging
from typing import Optional, Callable, Any

logger = logging.getLogger(__name__)

class ErrorRecovery:
    """Handle error recovery with fallback strategies"""
    
    @staticmethod
    def try_with_fallback(
        primary_func: Callable,
        fallback_func: Callable,
        operation_name: str
    ) -> Any:
        """Try primary operation, fallback if it fails"""
        try:
            logger.debug(f"Attempting primary operation: {operation_name}")
            return primary_func()
        except Exception as e:
            logger.warning(
                f"Primary operation failed: {operation_name}. "
                f"Falling back: {e}"
            )
            try:
                logger.debug(f"Attempting fallback operation: {operation_name}")
                return fallback_func()
            except Exception as fallback_error:
                logger.error(
                    f"Both primary and fallback failed for {operation_name}: "
                    f"Primary: {e}, Fallback: {fallback_error}",
                    exc_info=True
                )
                raise
    
    @staticmethod
    def try_with_default(
        func: Callable,
        default_value: Any,
        operation_name: str
    ) -> Any:
        """Try operation, return default if fails"""
        try:
            logger.debug(f"Attempting operation: {operation_name}")
            return func()
        except Exception as e:
            logger.warning(
                f"Operation failed, returning default: "
                f"{operation_name}, error: {e}"
            )
            return default_value

# Usage
def get_from_cache():
    raise ConnectionError("Cache unavailable")

def get_from_database():
    logger.info("Fetching from database")
    return "data_from_db"

# Try primary, fallback to secondary
result = ErrorRecovery.try_with_fallback(
    get_from_cache,
    get_from_database,
    "fetch_data"
)
print(f"Result: {result}")

# Try with default value
config = ErrorRecovery.try_with_default(
    lambda: int("invalid"),
    default_value=0,
    operation_name="parse_config"
)
print(f"Config: {config}")

## 9. Error Handler File Storage

import logging
import logging.handlers
import os
from datetime import datetime

class ErrorFileHandler(logging.handlers.RotatingFileHandler):
    """Custom file handler for error logging with automatic rotation"""
    
    def __init__(
        self,
        filename: str,
        max_bytes: int = 10*1024*1024,  # 10MB
        backup_count: int = 5,
        error_level: int = logging.ERROR
    ):
        super().__init__(filename, maxBytes=max_bytes, backupCount=backup_count)
        self.error_level = error_level
        
        # Detailed formatter for errors
        formatter = logging.Formatter(
            '[%(asctime)s] [%(levelname)s] [%(name)s] '
            '[%(filename)s:%(funcName)s:%(lineno)d] %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        self.setFormatter(formatter)
        self.setLevel(error_level)
    
    def emit(self, record: logging.LogRecord):
        """Override to add extra processing"""
        try:
            # Only emit errors and above
            if record.levelno >= self.error_level:
                super().emit(record)
        except Exception:
            self.handleError(record)

class ErrorHandlerConfig:
    """Configure error handlers for application"""
    
    @staticmethod
    def setup_error_handlers(log_dir: str = 'logs'):
        """Setup comprehensive error handling"""
        os.makedirs(log_dir, exist_ok=True)
        
        root_logger = logging.getLogger()
        root_logger.setLevel(logging.DEBUG)
        
        # Error handler - captures ERROR and CRITICAL
        error_handler = ErrorFileHandler(
            os.path.join(log_dir, 'errors.log'),
            error_level=logging.ERROR
        )
        root_logger.addHandler(error_handler)
        
        # Warning handler - captures WARNING and above
        warning_handler = logging.handlers.RotatingFileHandler(
            os.path.join(log_dir, 'warnings.log'),
            maxBytes=5*1024*1024,
            backupCount=3
        )
        warning_formatter = logging.Formatter(
            '[%(asctime)s] [%(levelname)s] %(message)s'
        )
        warning_handler.setFormatter(warning_formatter)
        warning_handler.setLevel(logging.WARNING)
        root_logger.addHandler(warning_handler)
        
        # Critical handler - captures only CRITICAL
        critical_handler = logging.FileHandler(
            os.path.join(log_dir, 'critical.log')
        )
        critical_formatter = logging.Formatter(
            '[%(asctime)s] CRITICAL: %(message)s | '
            '%(filename)s:%(funcName)s:%(lineno)d'
        )
        critical_handler.setFormatter(critical_formatter)
        critical_handler.setLevel(logging.CRITICAL)
        root_logger.addHandler(critical_handler)
        
        logger = logging.getLogger('app')
        logger.info(f"Error handlers configured. Log directory: {log_dir}")
        
        return root_logger

# Usage
if __name__ == "__main__":
    logger = logging.getLogger('app')
    ErrorHandlerConfig.setup_error_handlers()
    
    logger.info("Application starting")
    logger.warning("This is a warning")
    logger.error("This is an error")
    logger.critical("This is critical")

## 10. Best Practices Checklist

✅ **Do:**
- Use `logger.exception()` or `exc_info=True` for errors
- Create custom exception classes for your domain
- Log error context (IDs, parameters, state)
- Implement error recovery strategies
- Use separate error log files
- Structure logs for log aggregation
- Monitor error rates and types
- Clean up resources in finally blocks
- Use context managers for resource management
- Implement retry logic with backoff

❌ **Don't:**
- Ignore exceptions silently
- Log sensitive information (passwords, tokens)
- Create handlers inside try blocks
- Log the same error multiple times
- Use print() instead of logging
- Mix different log levels carelessly
- Lose error context when re-raising
- Forget to rotate log files
- Log sensitive stack traces to public logs
- Catch Exception without specific handling